<table style="width:100%"><tr>
<td> 
    
Technische Universität Berlin\
Electrical Engineering and Computer Science\
Internet of Things for Smart Buildings\
Prof. Dr. Sergio Lucia, Felix Fiedler, Benjamin Karg </td>
<td>  <img src="logo_tu.png" style="width: 20%;" align="right"/> </td>
</tr>
</table>

***
**MPC 20 - Exercise 03**
***

# <span class="graffiti-highlight graffiti-id_gyr7sdu-id_a518nym"><i></i>Please click here</span>!

# MPC implementation with CasADi

In this exercise we will continue working with the Python library [CasADi](https://web.casadi.org/docs/)
and use it to implement our own MPC. 
**We assume that you have completed the previous exercise in which we showed you the basics of CasADi.**
If you are working locally (on your computer) you need to have CasADi installed.


Import the required Python packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from casadi import *

We also change some Matplotlib parameters to increase the font size. Here is some [information](https://matplotlib.org/3.2.1/tutorials/introductory/customizing.html) on how this works.

In [ ]:
mpl.rcParams['font.size'] = 16

### <span class="graffiti-highlight graffiti-id_346f9jy-id_yxlygbd"><i></i>Define a linear system as a CasADi function</span>

In the following we investigate a simple linear system consisting of two linear oscillating masses which are connected through springs and can be controlled with a single input.
<img src="./figures/oscillating_masses.png" style="width: 100%;" align="center"/>

We define that linear system in discrete-time state-space form such that:

$$ x_{k+1} = Ax_{k}+Bu_{k}$$

where $x\in \mathbb{R}^n$, are the states of the system
and $u \in \mathbb{R}^m$ are the inputs.
The subscript $k$ and $k+1$ respectively, denote the discrete time instance of these variables.
In our case, the states are the displacement of the masses from their resting position as well as their velocity.


The state space system is defined in terms of the **system matrix** $A\in \mathbb{R}^{n\times n}$ and the **input matrix** $B\in \mathbb{R^{n\times m}}$. We skip the derivation and parameters of the system and just give you the numerical values of the resulting matrices.

In [ ]:
# System matrix:
A = np.array([[ 0.76272095,  0.45961393,  0.11486161,  0.0198116 ],
               [-0.89941626,  0.76272095,  0.41999073,  0.11486161],
               [ 0.11486161,  0.0198116 ,  0.76272095,  0.45961393],
               [ 0.41999073,  0.11486161, -0.89941626,  0.76272095]])
nx = A.shape[1]
print(A)
print('A.shape = {}'.format(A.shape))

In [ ]:
# The input matrix is defined as
B = np.array([[0.01413191],
            [0.06277108],
            [0.22062828],
            [0.36695456]])

nu = B.shape[1]
print(B)
print('B.shape = {}'.format(B.shape))

## <span class="graffiti-highlight graffiti-id_bnzxn8i-id_kfj9kex"><i></i>Task 1: Investigate the stability of the proposed system</span>

In a small preliminary task we investigate the stability property of the presented system.
For this purpose, we ask you to:

1. Determine the Eigenvalues of the system matrix. <br>
**On a sidenote**: You can trigger the help / doc string of any function simply by writing:
```
np.linalg.eig?
```
2. What is the stability region of time-discrete state-space systems?
3. Plot or compare the eigenvalues with the stability criteria.

In [ ]:
# Your code here!

<span class="graffiti-highlight graffiti-id_2osee49-id_9cfuiyr"><i></i><button>Show Solution</button></span>

The stability region of a discrete-time system is **within the unit circle**. As the eigenvalues lay directly on this circle, we can conclude that the system is stable but **not** asymptotically stable. 

## <span class="graffiti-highlight graffiti-id_0sqw2ph-id_clu8qo8"><i></i>Task 02: Create a symbolic expression of the state-space system</span>

We can also evaluate the state-space system symbolically:
1. Create symbolic variables ``x`` and ``u`` with their respective dimension. Note that we already created the variables ``nx`` and ``nu`` above.

In [ ]:
# Write your code here!

<span class="graffiti-highlight graffiti-id_3qgdiz3-id_rf22qp2"><i></i><button>Show Solution</button></span>

2. Create an symbolic expression for $x_{k+1}$ and a Function that takes two inputs ($x_k$, $u_k$) and returns $x_{k+1}$. This function should be named **system**:
```python
system = Function(...)
```

You may need to have a look at this list of [CasADi operations](https://web.casadi.org/docs/#list-of-operations) and the CasADi [function documentation](https://web.casadi.org/docs/#document-function). Additional CasADi operations
can be found [here](https://web.casadi.org/python-api/) (poorly documented). Note also that some numpy operations, such as ``np.sin`` can be used with CasADi symbolic variables. 


In [ ]:
# Write your code here!

<span class="graffiti-highlight graffiti-id_lprjo1o-id_qcnlkdq"><i></i><button>Show Solution</button></span>

3. Run the following code and see if it returns:
```
DM([1.37114, 0.460928, 1.57764, 0.765112])
```




In [ ]:
x0 = np.ones((4,1))
u0 = np.ones((1,1))
system(x0,u0)

**Just a small hint** for the implementation shown later (maybe this will not mean a anything to you just yet): You can also call functions with symbolic variables, e.g.:

In [ ]:
a = SX.sym("a", nx, 1)
b = SX.sym("b", nu, 1)

a_next = system(a,b)

print(a_next)

We have thus obtained the same expression as defined above but in terms of a new set of variables.

## <span class="graffiti-highlight graffiti-id_ck7ouy3-id_7kefe9q"><i></i>Task 03: Simulate a linear discrete-time system</span>
We first define the initial condition ``x_0`` and the control input ``u_k`` (constant for all times). 
We also initiate a new list to store the solution of the simulation, where the first element is ``x_0``.

The system will be simulated for ``N_sim=50`` steps.

In [ ]:
# Define the total steps to simulate
N_sim = 50
# Define the initial condition
x_0 = np.array([-2,2,1,-1]).reshape(4,1)
# Define the input (for the moment consider u = 0)
u_k = np.array([[0]])

res_x = [x_0]

1. Loop over ``i=0...N_sim`` and evaluate the update equation at each step.
2. Reset the initial state 
3. Save the current state by appending it to ``res_x``

4. After the loop has finished, concatenate the arrays in the list ``res_x``, such that you obtain an array of shape ``(nx, N_sim)``.

In [ ]:
# Write your code here!


<span class="graffiti-highlight graffiti-id_l8rwidz-id_bqi2n3d"><i></i><button>Show Solution</button></span>

We check the resulting dimension:

In [ ]:
res_x.shape

This should yield ``(4,51)``

## Plotting the results
We have prepared this code to plot your simulation results:

In [ ]:
fig, ax = plt.subplots(figsize=(10,6))

# plot the states
lines = ax.plot(res_x.T)
ax.legend(lines, ['mass 1 pos.', 'mass 1 vel.', 'mass 2 pos.', 'mass 2 vel.'])

# Set labels
ax.set_ylabel('states')
ax.set_xlabel('time')


As expected, the system is stable but not asymptotically stable.

# <span class="graffiti-highlight graffiti-id_v2py8q2-id_qc6m8mb"><i></i>MPC Tutorial</span>

We want to formulate the following discrete-time MPC problem:

\begin{align}
    & \min_{\begin{array}{c}
    x_0,\ \dots, \ x_{N+1},\\ 
    u_0,\ \dots,\ u_{N} \end{array}} \quad &\sum_{k=0}^N l(x_k,u_k) + m(x_{k+N}) \\
    \text{subject to:} \quad & x_0 = x_{\text{init}}, & \\
    & x_{k+1} = f(x_k,u_k) &,\\
    & x_{\text{lb}} \leq x_k \leq x_{\text{ub}}, &\, \forall k= 0,\dots, N+1, \\
    & u_{\text{lb}} \leq u_k \leq u_{\text{ub}}, &\, \forall k= 0,\dots, N, \\
\end{align}

The objective of the MPC controller is to regulate the system. Regulation means that the states are brought to zero. To achieve this goal we use a very common quadratic cost function, where for every time step $k$ of the horizon the following expression should be minimized:

$$l(x_k,u_k) = x_k^T Q x_k + u_k^T R u_k $$

where Q and R are positive definite diagonal matrices. 
For the terminal cost we have a slightly altered expression:

$$ m(x_{N+1}) = x_{N+1}^T P x_{N+1}, $$ 

since $u_{N+1}$ does not exist. **Note** that in this example we choose $P=Q$ for simplicity. 


Implementing and running the MPC controller for the previously introduced system and with the shown cost function can be roughly split into three steps. These steps are shown in the following.

## 1. Configuration for a generic stage 

The MPC optimization problem can be seen as having multiple stages corresponding to the time-steps of the prediction horizon. Stages are interconnected through the dynamic state equation and individually contribute to the MPC cost function. 
We advice to start the MPC implementation by configuring all components for a generic stage.
These components are 

- dynamic state equation
- the stage cost function (terminal cost function for the last state of the sequence)
- constraints for stage variables (upper and lower bounds in this example)

As you will see later, it is convenient to obtain CasADi functions for the first and second component. 
Since we have already created the **dynamic state equation in CasADi** above, it remains to obtain the **state and terminal cost functions** and to define the **upper and lower bounds for the optimization variables**.

## Cost function


### Tuning
Above we have formulated the cost function with arbitrary (positve definite) matrices. For the implementation you need to choose numerical values which will effect your controller performance. We provide here some values that yield a working controller but **you are invited to play with these values, once the algorithm is running.**

In [ ]:
Q = 20
Q = Q*np.diag(np.ones(nx))
print(Q)

In [ ]:
R = 10
R = np.diag(R*np.ones(nu))
print(R)

Another important parameter to choose is the prediction horizon. This determines "how far we look into the future". Note that an optimal solution does not guarante stability. **Feel free to test other values for the horizon**.

In [ ]:
N = 15

## Task 04: Construct CasADi objective function
Construct the cost function for the generic stage variables `x` and `u` and create CasADi functions from the resulting expressions. Also create the terminal cost as a function of `x`.

The resulting ``Function`` objects should be called 
- ``stage_cost_fcn``
- ``terminal_cost_fcn``

In [ ]:
# Write your code here!

<span class="graffiti-highlight graffiti-id_si47n14-id_dpqymsi"><i></i><button>Show Solution</button></span>

### Upper and lower bounds for stage variables:
These are suggested values that can be changed later.

In [ ]:
# state constraints
lb_x = -4*np.ones((nx,1))
ub_x = 4*np.ones((nx,1))
# input constraints
lb_u = -1.5*np.ones((nu,1))
ub_u = 1.5*np.ones((nu,1))

## <span class="graffiti-highlight graffiti-id_ostqpeu-id_0yfll0e"><i></i>2. Create the optimization problem</span>

The second step of the process is already to create the optimization problem. Note that we want to end up with something like this:

$$
\min_{x} \quad J(x)\\
\text{s.t.} \quad g_{lb}\leq g(x)\leq g_{ub}\\
\quad \ x_{lb}\leq x \leq x_{ub}
$$

We thus need to create the cost function $J(x)$ and the constraint functions $g(x)$. But first of all, **we need to create our optimization variables**. These are not the previously introduced

```python
x = SX.sym("x",nx,1)
u = SX.sym("u",nu,1)
```

because we are optimizing over **sequences of states and inputs**.

## Define optimization variables

Note that the upper cases variables ``X`` and ``U`` are stacked vectors that **contain the states and inputs for all time-steps**. Also note that ``X`` contains one additional sequence element than ``U``.

In [ ]:
X = SX.sym("X",(N+1)*nx,1)
U = SX.sym("U",N*nu,1)

## Task 05: Formulate MPC optimization problem
We are now prepared to formulate the MPC optimization problem, which consists of the cost ``J
``, the constraints ``g`` and the bounds. We start by initializing these variables:

In [ ]:
J = 0
lb_X = [] # lower bound for X.
ub_X = [] # upper bound for X
lb_U = [] # lower bound for U
ub_U = [] # upper bound for U
g = []    # constraint expression g
lb_g = []  # lower bound for constraint expression g
ub_g = []  # upper bound for constraint expression g

**The idea is now the following**. We are looping over the different stages of our prediction horizon:

```python
for k in range(N):
    ...
```

and at each iteration we append the stage:
- cost
- constraints
- bounds

in terms of the stage variables ``x_k`` and ``u_k``. 
Fortunately, we have already defined the functions:

- ``stage_cost_fcn(x_k,u_k)`` (stage cost)
- ``system(x_k,u_k)`` (dynamic state equation)
- ``terminal_cost_fcn(x_N+1)`` (terminal cost)

which now come very handy! 

The **dynamic state equation** is enforced by adding the following constraint at each stage:
```
g_k = f(x_k, u_k) - x_{k+1}
```

Since the stage variables ``x_k`` and ``u_k`` are part of the stacked vectors ``X`` and ``U``,
we first have to retrieve them by slicing.

The full loop to create the optimization problem must thus look something like this:

```python
for k in range(N):
    # Use slicing to obtain stage variables
    x_k = X[]
    u_k = U[]
    
    ...
    
    # Cost
    J += stage_cost
    
    # constraints
    g.append(stage_constraint)
    
    # bounds
    lb_X.append(lb_x)
    ...
```

With this idea in mind, complement the following code.
We have numbered sections corresponding to their respective sub tasks as shown below:

```
# 01 - Your code here!
x = 
...
# 01
```

### Your tasks:
1. Use slices to query the following elements of ``X`` and ``U`` at the different stages:
  - ``x_k``
  - ``x_k_next``
  - ``u_k``
2. Add the cost of the current stage to the stage cost.
3. Evaluate the system equation **symbolically** to determine ``x_k_next_calc``.
4. Add the equality constraint that ``x_k_next = x_k_next_calc``. Append the expression to ``g`` and update ``lb_g``, ``ub_g`` accordingly.
5. Append the upper and lower bounds for ``x_k`` and ``u_k`` to ``lb_X``, ``ub_X``.

In [ ]:
# formulate optimization problem

for k in range(N):
    # 01 - Your code here!

    

    # 02 - Your code here!

    
    
    # 03 - Your code here!



    # 04 - Your code here!



    # 05 - Your code here!



<span class="graffiti-highlight graffiti-id_32pqtm0-id_kyb7tut"><i></i><button>Show Solution</button></span>

6. Finally, you need to include the constraints and terminal cost for the last state of the sequence. 
Since we stop at this stage there is no need to consider the dynamic state equation again.

In [ ]:
# Your code here!

<span class="graffiti-highlight graffiti-id_eyn1bt9-id_13rvrth"><i></i><button>Show Solution</button></span>

## <span class="graffiti-highlight graffiti-id_c3vg46h-id_plnfws9"><i></i>Task 06: Create the CasADi solver for the optimization problem</span>
To solve the problem defined above, we first need to create a CasADi ``nlpsol`` object, where an optimization problem is defined as shown here:

$$
\min_{x} \quad J(x)\\
\text{s.t.} \quad g_{lb}\leq g(x)\leq g_{ub}\\
\quad \ x_{lb}\leq x \leq x_{ub}
$$

Note that CasADi expects a single vector ``x`` as the optimization variables with corresponding upper and lower bound. 
You should thus:

- concatenate ``X`` and ``U`` to a single optimization variable.
- concatenate the bounds 
- Create the CasADi ``nlpsol`` object as described [here](https://web.casadi.org/docs/#nonlinear-programming).

**Note**: You will need the CasADi ``vertcat`` function which takes an arbitrary number of inputs. You can use the **splat** operator (*) to pass multiple inputs to a function which originate from a list, e.g.:

```python
vertcat(a,b,c,d)
# or 
vertcat(*[a,b,c,d])
# or 
vertcat(*[a,b],*[c,d])
```
whereas:
```python
vertcat([a,b,c,d])
```
will throw an error.

In [ ]:
# Your code here!

<span class="graffiti-highlight graffiti-id_gb1od5w-id_57awt6v"><i></i><button>Show Solution</button></span>

# 3. <span class="graffiti-highlight graffiti-id_imi5mp0-id_3k6fiej"><i></i>MPC loop</span>

Now that we have the configured solver, we can see how the MPC controller performs in action. The procedure of the main loop is illustrated in this figure:

<img src="./figures/mpc_loop.svg" style="width: 80%;" align="center"/>

At each iteration we have to call the solver which has the following inputs:

In [ ]:
print(solver)

- **x0:** This is not the state but the initial guess for the optimization variables (``X`` and ``U``). The initial guess is optional but in many cases crucial for the success of the solver.
- **lbx:** Lower bounds to both states and control inputs. These are defined already.
- **ubx:** Upper bounds to both states and control inputs. These are defined already.
- **lbg:** Lower bounds for expression ``g``. As ``g`` contains only equality constraints, both upper and lower bounds are set to the same value.
- **ubg:** Upper bounds for expression ``g``. As ``g`` contains only equality constraints, both upper and lower bounds are set to the same value.


**Note**: As is common practice, we use the constraints ``lbx``and ``ubx`` to enforce the initial state, **which is reset at every MPC iteration**. ``lbx``and ``ubx`` are the concatenated bounds for all $x_k$ and $u_k$.


## Task 07: Run the solver and investigate the output

Before we run the final loop, let's just call the solver with some sane inputs and investigate the solution.

- fix the initial state by using the optimization variable bounds
- run the solver. Pass at least ``lbx, ubx, lbg, ubg``.
- extract the sequence of states ``X`` as an ``(N+1, n_x)`` array
- extract the sequence of states ``U`` as an ``(N, n_u)`` array

**Note**, the call:
```python 
res = solver( ... ) 
```

returns a dictionary, with the following fields:
- ``f`` - the optimal cost
- ``g`` - the values of the constraints
- ``lam_g`` - the lagrange multipliers for ``g``
- ``lam_x`` - the lagrange multipliers for ``x``
- ``x`` - the optimal solution

You usually want to extract ``res['x']``.

In [ ]:
x_0 = np.array([-2,2,1,-1]).reshape(4,1)

# Write your code here!

<span class="graffiti-highlight graffiti-id_q9mu7j4-id_jhna2e1"><i></i><button>Show Solution</button></span>

Please make sure that the solver returned with:

```
EXIT: Optimal Solution Found.
```
otherwise we cannot expect anything from the solution!



### <span class="graffiti-highlight graffiti-id_obt66k0-id_w78d8lw"><i></i>Visualizing the prediction</span>
Below we plot the predicted states and inputs. 
We can see that the optimizer finds a solution that drives the states slowly to zero.
Furthermore, we observe that the constraints (visible for the inputs) are satisfied. 
Please also validate that your initial state is as defined.

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10,8), sharex=True)
ax[0].plot(X)
ax[1].plot(U)
ax[0].set_ylabel('states')
ax[1].set_ylabel('control input')
ax[1].set_xlabel('time')

# Highlight the selected initial state (the lines should start here!)
ax[0].plot(0,x_0.T, 'o', color='black')

fig.align_ylabels()
fig.tight_layout()

## Task 08: Create and run MPC main loop
Now that everything is working, we can finally run the MPC loop.
At each iteration:

- fix the initial state by using the optimization variable bounds
- run the solver. Pass at least ``lbx, ubx, lbg, ubg``.
- extract the **current** control input from the optimal solution
- similarly to **Task 03:** Simulate the system (with the obtained control input) and obtain the next state
- store the current state and solution to ``res_x`` and ``res_u``
- reset the initial state to the next state

In [ ]:
x_0 = np.array([-2,2,1,-1]).reshape(4,1)
res_x = [x_0]
res_u = []

N_sim = 50

# Write your code here:

<span class="graffiti-highlight graffiti-id_4t5oguw-id_8t63dla"><i></i><button>Show Solution</button></span>

### Plot the results
We have again prepared a plot for you to visualize the obtained solution.

In [ ]:
fig, ax = plt.subplots(2,1, figsize=(10,8), sharex=True)
ax[0].plot(res_x.T)
ax[1].plot(res_u.T)
ax[0].set_ylabel('states')
ax[1].set_ylabel('control input')
ax[1].set_xlabel('time')

fig.align_ylabels()
fig.tight_layout()

Congratulations, the system appears to be stable now! Also note that the optimal solution respects your choosen constraints with respect to the control inputs. 

You have now implemented your first MPC scheme!

## Next steps ...

This week we have implemented a very simple MPC scheme that has a couple of shortcomings in view of your own projects. Most notably, we have worked with a discrete-time system whereas many of the presented projects will require you to tackle continuous time systems. We will thus **study discretization schemes next week** to approach the continuous-time case. Most importantly, we will present **orthogonal collocation on finite elements**.